In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Exploration

- App: The application name.
- Category: The category the app belongs to.
- Rating: Overall user rating of the app.
- Reviews: Number of user reviews for the app.
- Size: The size of the app.
- Installs: Number of user installs for the app.
- Type: Either "Paid" or "Free".
- Price: The price of the app.

- Content Rating: The age group the app is targeted at - "Children" / "Mature 21+" / "Adult".
- Genres: Possibly multiple genres the app belongs to.
- Last Updated: The date the app was last updated.
- Current Ver: The current version of the app.
- Android Ver: The Android version is needed for this app.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from utils import data
from dateutil.parser import parse

In [ ]:
data.head()

In [ ]:
# On remarque que la première colonne n'a pas de nom.
data.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
data.columns

In [ ]:
data.shape

## Verification de l'integrité des données

In [ ]:
# Est-ce qu'il y a des nulls ?
data.isnull().sum()

In [ ]:
# Combien de valeurs uniques par colonne ?
for col in data.columns:
    print(f"{col}:")
    unique = data[col].unique()
    unique_length = unique.size
    print(f"\t Unique count: {unique_length}")
    # Et pour ceux avoir moins de 50 valeurs on les affiches
    if unique_length > 50:
        print("\t Too Many Values")
    else:
        print(f"\t {unique}")
    print()
    

On constate qu'une ligne a ses colonnes est décalée


In [ ]:
data[data['Installs'] == 'Free']

---

# Nettoyage des données

Suppression de la ligne dont les colonnes sont décalées

In [ ]:
data = data[data['Installs'] != 'Free']

Transformation de la colonne "Last Updated" en date

In [ ]:
data["Last Updated"] = data["Last Updated"].apply(lambda x: parse(x))
data.dtypes

Créer une colonnes avec le nombre d'installation en nombre

In [ ]:
data['Installs'] = data['Installs'].str.replace('[+,]', '', regex=True).astype(int)
data['Installs'].unique()

In [ ]:
data['Price'] = data['Price'].str.replace('[$]', '', regex=True).astype(float)

Cast du type de Reviews en int

In [ ]:
data['Reviews'] = data['Reviews'].map(int)

Ajout d'une colonne représentant le nombre d'utilisateurs n'ayant pas attribué de note.

In [ ]:
data['Unreviewed'] = data['Installs'] - data['Reviews']
data['Unreviewed'].size

---

# Graphes

## Moyenne du nombre de téléchargements par catégorie d'applications

Préparation des données

In [ ]:
# Calculer le nombre total de téléchargements par catégorie
total_downloads_by_category = data.groupby('Category')['Installs'].sum().sort_values()
# Calculer le nombre total d'applications par catégorie
total_apps_by_category = data['Category'].value_counts()
# Calculer la moyenne des téléchargements par catégorie
average_downloads_by_category = total_downloads_by_category / total_apps_by_category
# Trier les catégories d'applications par ordre croissant de moyenne de téléchargements
average_downloads_by_category = average_downloads_by_category.sort_values(ascending=True)

Création du graphe

In [ ]:
fig = go.Figure(data=[go.Bar(
    x=average_downloads_by_category.index,
    y=average_downloads_by_category
)])
fig.update_layout(
    title_text='Moyenne du nombre de téléchargements par catégorie d\'applications',
    xaxis_title="Catégorie", 
    yaxis_title="Moyenne du nombre de téléchargements", 
    width=900,
    xaxis=dict(tickmode = 'linear', dtick = 1),
    title_x=0.5
)
;

In [ ]:
fig.show()

# Nombre d'application gratuite et payante

Préparation des données

In [ ]:
types = data["Type"]
type_counts = types.value_counts()

Création du graphe

In [ ]:
fig = go.Figure(data=[go.Pie(
    labels=type_counts.index,
    values=type_counts
)])
fig.update_layout(
    title_text='Nombre d\'application gratuite et payante',
    width=500,
    title_x=0.5
)
;

In [ ]:

fig.show()

# Nombre d'applications par catégorie

Obtention du nombre d'applications par catégorie

In [ ]:
category = data['Category'].value_counts()

Affichage du nombre d'applications par catégorie dans un graphique à barres horizontales

In [ ]:
fig = go.Figure(data=[go.Bar(
    y=category.index,
    x=category,
    orientation='h'
)])
fig.update_layout(
    title_text='Nombre d\'applications par catégorie',
    xaxis_title="Nombre d'applications", 
    width=900,
    title_x=0.5
)
;

In [ ]:
fig.show()

# Meilleures catégories par note sur le contenu

Préparation des données

In [ ]:
# Groupe les valeurs par note de sur le contenu
grouped_categories = data.groupby('Content Rating')['App'].count()
top_categories = grouped_categories.sort_values(ascending=False)

Création du graphe

In [ ]:
fig = go.Figure(data=[go.Pie(
    labels=top_categories.index,
    values=top_categories
)])
fig.update_layout(
    title_text='Meilleures catégories par note sur le contenu',
    width=500,
    title_x=0.5
)
;

In [ ]:
fig.show()

# Nombre d\'application par note

In [ ]:
ratings_count = data['Rating'].value_counts()

In [ ]:
fig = go.Figure(data=[go.Bar(
    x=ratings_count.index,
    y=ratings_count
)])
fig.update_layout(
    title_text='Nombre d\'application par note',
    xaxis_title="Note", 
    yaxis_title="Nombre d'application à cette note", 
    width=900,
    xaxis=dict(tickmode = 'linear', dtick = 1),
    title_x=0.5
)
;

In [ ]:
fig.show() 

# Pourcentage d'avis posté comparé au nombre d'installations

In [ ]:
# Groupe les valeurs par note de sur le contenu
reviews_and_unreviewed = data[['Reviews', 'Unreviewed']].sum()

In [ ]:
fig = go.Figure(data=[go.Pie(
    labels=reviews_and_unreviewed.index,
    values=reviews_and_unreviewed,
    
)])
fig.update_layout(
    title_text='Pourcentage d\'avis posté comparé au<br>nombre d\'installations',
    width=500,
    title_x=0.5
)
;

In [ ]:
fig.show()